In [1]:
%load_ext autoreload
%autoreload 2
# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os
from coeditor.retrieval_model import RetrievalEditorModel, AttentionMode
from coeditor.api import EditPredictionService, QueryRefEditEncoder, BatchArgs, DecodingArgs

os.chdir(proj_root())

In [3]:
# target_dir = proj_root() / "../SPOT-copy/"
target_dir = proj_root()
model_name = "coeditor-xl-bi-request-stub-comments-v4"
model_path = get_model_dir(True) / model_name
model = RetrievalEditorModel.load(model_path)
model.to("cuda:2")
batch_args = BatchArgs.service_default()
service = EditPredictionService(
    target_dir,
    model,
    batch_args=batch_args,
    encoder=QueryRefEditEncoder(
        max_ref_tks=batch_args.max_ref_tks,
        max_query_tks=batch_args.max_query_tks,
        max_output_tks=batch_args.max_output_tks,
    ),
    # dec_args = DecodingArgs(do_sample=False, num_beams=8, length_penalty=0.0)
    dec_args = DecodingArgs(do_sample=True, top_p=0.95, marginalize_samples=50)
)


In [4]:
model.push_to_hub(f"MrVPlusOne/{model_name}")

CommitInfo(commit_url='https://huggingface.co/MrVPlusOne/coeditor-xl-bi-request-stub-comments-v4/commit/23e44770495c87f5a13acb28c6e6885b970e1401', commit_message='Upload model', commit_description='', oid='23e44770495c87f5a13acb28c6e6885b970e1401', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
service.dec_args = DecodingArgs(do_sample=True, top_p=0.9, marginalize_samples=25)
# service.dec_args = DecodingArgs(do_sample=False, num_beams=8, length_penalty=0.0)
service.tlogger.clear()
response = service.suggest_edit(Path("src/coeditor/retrieval_model.py"), 463)
print(response)
service.tlogger.as_dataframe()

respect_lines = 71
stub files: dict_keys(['coeditor.ctx_change_encoder', 'spot.static_analysis', 'coeditor.retrieval_model'])


edits_to_batches: 100%|██████████| 1/1 [00:00<00:00, 7973.96it/s]


num batches: 1, batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}


  7%|▋         | 38/512 [00:02<00:31, 15.13it/s, unfinished=0] 


Writing logs to: .coeditor_logs
Target file: /home/jiayi/Projects/SPOT/src/coeditor/retrieval_model.py
Edit range: (387, 0) - (485, 24)
	--------------- Suggestion 0 (score: 0.52) ---------------
	... (no change) ...
	--------------- Suggestion 1 (score: 0.12) ---------------
	... 6 lines omitted...
	                  change = req.target.map(lambda x: x.code)
	                  change_tks = change_to_tokens(change)
	                  pred = apply_output_tks_to_change(change_tks, req.respect_lines, out)
	                  pred_changes.append(pred)
	          assert_eq(len(pred_changes), len(out_tks), len(pred_scores))
 
	-         solutions = list[list[PredictedChange]]()
	+         solutions = list[tuple[str, PredictedChange]]()
	          for i in range(0, len(pred_changes), N):
	              sols = marginalize_preds(
	                  pred_changes[i : i + N],
	                  out_tks[i : i + N],
	                  pred_weights[i : i + N],
	                  pred_scores[i : i + N]

,name,count,avg_time,total_time
6,run model,1,2.967389,2.967389
1,construct project edit,1,2.679183,2.679183
4,model.generate,1,2.514444,2.514444
2,encode edits,1,0.926231,0.926231
0,get target element,1,0.556301,0.556301
3,decoder.forward,38,0.013228,0.502646
5,assemble changes,1,0.414982,0.414982


In [19]:
import json
print(json.dumps(response.to_json(), indent=2))

{
  "target_file": "/home/jiayi/Projects/SPOT/src/coeditor/api.py",
  "edit_start": [
    235,
    4
  ],
  "edit_end": [
    373,
    9
  ],
  "old_code": "def suggest_edit(\n        self,\n        file: Path,\n        line: int,\n        log_file: Path | None = Path(\"coeditor-log.txt\"),\n    ) -> ServiceResponse:\n        \"\"\"Make the suggestion in-place at the given location.\"\"\"\n        timed = self.tlogger.timed\n        project = self.project\n\n        if not file.is_absolute():\n            file = project / file\n\n        with timed(\"get target element\"):\n            mname = PythonProject.rel_path_to_module_name(file.relative_to(project))\n            stamp = os.stat(file).st_mtime\n            now_code = file.read_text()\n            now_mod = self.parse_cache.cached(\n                mname,\n                stamp,\n                lambda: PythonModule.from_cst(\n                    cst.parse_module(now_code), mname, drop_comments=False\n                ),\n        

In [4]:
service.suggest_edit(Path("src/coeditor/common.py"), 294)

stub files: dict_keys(['coeditor.api', 'coeditor.common', 'coeditor.encoders', 'coeditor.retrieval_model'])
num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
========== Sugeestion 0 ==========
score: -11.35
* Modified: 
        # edit: coeditor.common/random_subset
    -   def random_subset(all, n: int, seed: int = 42):
    +   def random_subset(all, n: int, rng: random.Random | None = None):
    +       if rng is None:
    +           rng = random.Random(42)
            if isinstance(all, Sequence):
                xs = [x for x in all]
    -           random.Random(seed).shuffle(xs)
    +           rng.shuffle(xs)
                return xs[:n]
            elif isinstance(all, Mapping):
                keys = [k for k in all]
    -           random.Random(seed).shuffle(keys)
    +           rng.shuffle(keys)
                return {k: all[k] for k in keys[:n]}
            else:
                raise ArgumentError(all, f"Unsupported arg type: {typ

In [5]:
service.suggest_edit(Path("src/coeditor/encoders.py"), 292)

stub files: dict_keys(['coeditor.api', 'coeditor.common', 'coeditor.encoders', 'coeditor.retrieval_model'])
num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
========== Sugeestion 0 ==========
score: -2.132
* Modified: 
        # edit: coeditor.encoders/has_change
    -   def has_change(tks: TokenSeq) -> bool:
    -       return Add_id in tks or Del_id in tks
    +   def has_change(tks: TokenSeq, skip_prefix: int = 16) -> bool:
    +       return Add_id in tks or Del_id in tks[skip_prefix:]
========== Sugeestion 1 ==========
score: -3.006
* Modified: 
        # edit: coeditor.encoders/has_change
    -   def has_change(tks: TokenSeq) -> bool:
    -       return Add_id in tks or Del_id in tks
    +   def has_change(tks: TokenSeq, skip_prefix: int = 16) -> bool:
    +       return Add_id in tks or Del_id in tks or skip_prefix in tks
========== Sugeestion 2 ==========
score: -3.18
* Modified: 
        # edit: coeditor.encoders/has_change
    -   def h

In [6]:
service.tlogger.as_dataframe()


,name,count,avg_time,total_time
3,run model,3,2.283796,6.851387
1,construct project edit,3,0.674862,2.024587
0,get target element,3,0.257443,0.772328
2,encode edits,3,0.151020,0.453060
